In [0]:

import re

letras_posibles="TRWAGMYFPDXBNJZSQVHLCKE"

class DNI:

    def __init__(self, dni_como_texto):
        self.dni_como_texto = self.pretratamiento_de_dni(dni_como_texto)

        self.confianza = 0
        (valido, numero, letra) = self.validar_dni(self.dni_como_texto)
        if(valido):
            self.confianza = 1
        else:
            (valido, numero, letra) = self.tratar_de_arreglar_el_dni()
            if(valido):
                self.confianza = 0.5
        self.valido = valido
        self.letra = letra
        self.numero = numero

    def letra_correspondiente(self,numero_dni):
        return letras_posibles[numero_dni % 23]

    def validar_dni(self, dni_como_texto):
        """                             VALIDACION                    -->   FORMATEO
        Me llegará algo como:       NUMERO          LETRA     VALIDO
            12345678T           ->  12345678        T           √           --> 12345678-T | 12345678T | 12345678 T | 12.345.678-T
            12345.678-T             12345678        T           x
            12.345.678 T            12345678        T           √
            00001234T               1234            T           x
            1234t                   1234            T           x
        """
        numero = None
        letra = None
        # Aplico una expresion regular
        valido = re.match(r'^[0-9]{1,8}[ -]?[a-zA-Z]$', dni_como_texto)
        if(valido):
            valido = True
            # Extraer la letra 
            letra = self.dni_como_texto[-1].upper()
            # Extraer el número
            dni_como_texto = dni_como_texto.replace(". -","") # Me como los espacios, puntos y guiones
            numero = int(dni_como_texto[:-1])
            # Validar la letra
            if self.letra_correspondiente(numero) != letra:
                valido = False
        else:
            valido = False
        return (valido, numero, letra)

    def tratar_de_arreglar_el_dni(self):
        """
            Si me vienen puntos en mal sitio, lo ignoro...                          [^a-zA-Z0-9]
            Si me viene algun caracter que no sea letras o numeros... los ignoro
        """
        dni_como_texto = re.sub(r'[^a-zA-Z0-9]','',self.dni_como_texto)
        print(dni_como_texto)
        return self.validar_dni(dni_como_texto)

    def pretratamiento_de_dni(self,dni_como_texto):
        return dni_como_texto.strip()

In [0]:
dni_como_texto = """ 230.0.00$T
"""

dni = DNI(dni_como_texto)
# Acabao!
# Si tengo confianza al menos 0.5, lo formateo como me interese para los cruces con otra BBDD
#if(confianza >= 0.5):
#    dni_formatado_para_cruces = formatear_dni(letra, numero, puntos= False, separado_letra = '', rellenar_con_ceros = True)
print(dni.valido, dni.confianza, dni.letra, dni.numero, dni.dni_como_texto)



230000T
True 0.5 T 230000 230.0.00$T


In [0]:

import re

letras_posibles="TRWAGMYFPDXBNJZSQVHLCKE"

class DNI:

    def __init__(self, dni_como_texto):
        self.dni_como_texto = self.pretratamiento_de_dni(dni_como_texto)

        self.confianza = 0
        (valido, numero, letra) = self.validar_dni(self.dni_como_texto)
        if(valido):
            self.confianza = 1
        else:
            (valido, numero, letra) = self.tratar_de_arreglar_el_dni()
            if(valido):
                self.confianza = 0.5
        self.valido = valido
        self.letra = letra
        self.numero = numero

    def letra_correspondiente(self,numero_dni):
        return letras_posibles[numero_dni % 23]

    def validar_dni(self, dni_como_texto):
        numero = None
        letra = None
        valido = re.match(r'^[0-9]{1,8}[ -]?[a-zA-Z]$', dni_como_texto)
        if(valido):
            valido = True
            letra = self.dni_como_texto[-1].upper()
            dni_como_texto = dni_como_texto.replace(". -","") # Me como los espacios, puntos y guiones
            numero = int(dni_como_texto[:-1])
            if self.letra_correspondiente(numero) != letra:
                valido = False
        else:
            valido = False
        return (valido, numero, letra)

    def tratar_de_arreglar_el_dni(self):
        dni_como_texto = re.sub(r'[^a-zA-Z0-9]','',self.dni_como_texto)
        print(dni_como_texto)
        return self.validar_dni(dni_como_texto)

    def pretratamiento_de_dni(self,dni_como_texto):
        return dni_como_texto.strip()

In [0]:
ruta_archivo = "/tmp/personas.json"

dbutils.fs.put(ruta_archivo, """
{"nombre":"Juan","apellidos":"García","edad":25, "dni": "23000000T", "cp": "28001", "email": "juan@garcia.es"}
{"nombre":"Juan","apellidos":"Ruiz","edad":17, "dni": "23000023T", "cp": "28006", "email": "juan@ruiz.es"}
{"nombre":"María","apellidos":"García","edad":43, "dni": "23000046T", "cp": "28006", "email": "maria@garcia.com"}
{"nombre":"Menchu","apellidos":"Ruiz","edad":33, "dni": "23000002T", "cp": "28002", "email": "menchu@ruiz.net"}
""", True)


Wrote 448 bytes.


True

In [0]:
# Quiero que spark lea ese fichero.
# Quien me hace eso guay es la librería spark-sql

# Para trabajar con esta librería no usamos la palabra/variable "sc"
# La variable "sc" me da una conexión con un cluster de spark para trabajar con la libraria spark-core
# En este caso, para forzar el trabajo con la libreria spark-sql, usamos la variable "spark"

datos = spark.read.json(ruta_archivo)
datos # Básicamente lo que muestra es el ESQUEMA que tiene asociado ese conjunto de datos.
            # El esquema define los campos de cada fila del DataFrame y sus tipos de datos (también si son nulables)
            # Es el mismo concepto que el esquema de una tabla de una BBDD
            # El esquema se me ha generado en automático... desde los datos... NORMALMENTE esto me vale...


DataFrame[apellidos: string, cp: string, dni: string, edad: bigint, email: string, nombre: string]

In [0]:
datos.printSchema()

# miDataframe.schema[0].nullable = False EDITAR EL ESQUEMA

root
 |-- apellidos: string (nullable = true)
 |-- cp: string (nullable = true)
 |-- dni: string (nullable = true)
 |-- edad: long (nullable = true)
 |-- email: string (nullable = true)
 |-- nombre: string (nullable = true)



In [0]:
datos.show()

+---------+-----+---------+----+----------------+------+
|apellidos|   cp|      dni|edad|           email|nombre|
+---------+-----+---------+----+----------------+------+
|   García|28001|23000000T|  25|  juan@garcia.es|  Juan|
|     Ruiz|28006|23000023T|  17|    juan@ruiz.es|  Juan|
|   García|28006|23000046T|  43|maria@garcia.com| María|
|     Ruiz|28002|23000002T|  33| menchu@ruiz.net|Menchu|
+---------+-----+---------+----+----------------+------+



In [0]:
datos.select("nombre", "apellidos").show() # SELECT nombre, apellidos FROM datos

+------+---------+
|nombre|apellidos|
+------+---------+
|  Juan|   García|
|  Juan|     Ruiz|
| María|   García|
|Menchu|     Ruiz|
+------+---------+



In [0]:
from pyspark.sql.functions import col
# En estas funciones, podemos pasar dentro de los parentesis:
# - Nombres de las columas
# - Pasar objetos de tipo COLUMNA, que son un tipo de objetos que tiene spark sql

datos.select( col("nombre"), col("apellidos")).show() # SELECT nombre, apellidos FROM datos

+------+---------+
|nombre|apellidos|
+------+---------+
|  Juan|   García|
|  Juan|     Ruiz|
| María|   García|
|Menchu|     Ruiz|
+------+---------+



In [0]:
# Las columas de spark ofrecen funciones adicionales
datos.select( col("nombre"), col("apellidos"), col("edad") >=18 ).show() # SELECT nombre, apellidos FROM datos

# Esas 2 sintaxis no se pueden MEZCLAR !!!!

+------+---------+------------+
|nombre|apellidos|(edad >= 18)|
+------+---------+------------+
|  Juan|   García|        true|
|  Juan|     Ruiz|       false|
| María|   García|        true|
|Menchu|     Ruiz|        true|
+------+---------+------------+



In [0]:
# Igual que la función select, tengo muchas otras... similares a las de SQL
datos.orderBy("edad").show()

+---------+-----+---------+----+----------------+------+
|apellidos|   cp|      dni|edad|           email|nombre|
+---------+-----+---------+----+----------------+------+
|     Ruiz|28006|23000023T|  17|    juan@ruiz.es|  Juan|
|   García|28001|23000000T|  25|  juan@garcia.es|  Juan|
|     Ruiz|28002|23000002T|  33| menchu@ruiz.net|Menchu|
|   García|28006|23000046T|  43|maria@garcia.com| María|
+---------+-----+---------+----+----------------+------+



In [0]:
datos.orderBy(col("edad").desc()).show()

+---------+-----+---------+----+----------------+------+
|apellidos|   cp|      dni|edad|           email|nombre|
+---------+-----+---------+----+----------------+------+
|   García|28006|23000046T|  43|maria@garcia.com| María|
|     Ruiz|28002|23000002T|  33| menchu@ruiz.net|Menchu|
|   García|28001|23000000T|  25|  juan@garcia.es|  Juan|
|     Ruiz|28006|23000023T|  17|    juan@ruiz.es|  Juan|
+---------+-----+---------+----+----------------+------+



In [0]:
datos.select("nombre","edad") \
     .filter(col("edad")>=18) \
     .groupBy("nombre") \
     .sum("edad") \
     .orderBy("sum(edad)") \
     .show()
    # Intermnamente, la libreria spark sql transforma todas estas operaciones en operaciones MAP-REDUCE de la libreria core

+------+---------+
|nombre|sum(edad)|
+------+---------+
|  Juan|       25|
|Menchu|       33|
| María|       43|
+------+---------+



In [0]:
datos.createOrReplaceTempView("personas")
otro_dataframe=spark.sql("SELECT nombre, sum(edad) FROM personas WHERE edad >= 18 GROUP BY nombre ORDER BY sum(edad) DESC")
otro_dataframe.show()

+------+---------+
|nombre|sum(edad)|
+------+---------+
| María|       43|
|Menchu|       33|
|  Juan|       25|
+------+---------+

